## Deliverable 2. Create a Customer Travel Destinations Map.

In [92]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
config = "../World_Weather_Analysis/config.py"
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [93]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,73.54,89,72,5.99,broken clouds
1,1,Avarua,CK,-21.2078,-159.7750,80.65,74,23,12.66,few clouds
2,2,Bengkulu,ID,-3.8004,102.2655,80.98,81,91,1.57,overcast clouds
3,3,Castro,BR,-24.7911,-50.0119,58.26,93,81,3.51,broken clouds
4,4,Dingle,PH,10.9995,122.6711,86.92,70,75,6.04,light rain


In [94]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [80]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Galesong,ID,-5.3166,119.3661,75.22,72,54,3.33,broken clouds
2,2,Conakry,GN,9.5380,-13.6773,84.27,65,40,6.91,scattered clouds
3,3,Alepe,CI,5.5004,-3.6631,82.27,94,72,3.36,broken clouds
5,5,Gigmoto,PH,13.7796,124.3900,85.35,71,84,2.62,broken clouds
8,8,Conceicao Do Araguaia,BR,-8.2578,-49.2647,82.53,61,0,3.98,clear sky
9,9,Hithadhoo,MV,-0.6000,73.0833,83.64,70,100,17.36,overcast clouds
15,15,Kapaa,US,22.0752,-159.3190,82.72,78,90,4.00,light rain
17,17,Butaritari,KI,3.0707,172.7902,83.26,74,94,10.25,overcast clouds
22,22,Kahului,US,20.8947,-156.4700,85.96,68,40,26.46,scattered clouds
24,24,Vaini,TO,-21.2000,-175.2000,77.16,83,20,2.30,few clouds


In [81]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
dtype: int64

In [82]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Galesong,ID,-5.3166,119.3661,75.22,72,54,3.33,broken clouds
2,2,Conakry,GN,9.5380,-13.6773,84.27,65,40,6.91,scattered clouds
3,3,Alepe,CI,5.5004,-3.6631,82.27,94,72,3.36,broken clouds
5,5,Gigmoto,PH,13.7796,124.3900,85.35,71,84,2.62,broken clouds
8,8,Conceicao Do Araguaia,BR,-8.2578,-49.2647,82.53,61,0,3.98,clear sky


In [83]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Galesong,ID,75.22,broken clouds,-5.3166,119.3661,
2,Conakry,GN,84.27,scattered clouds,9.5380,-13.6773,
3,Alepe,CI,82.27,broken clouds,5.5004,-3.6631,
5,Gigmoto,PH,85.35,broken clouds,13.7796,124.3900,
8,Conceicao Do Araguaia,BR,82.53,clear sky,-8.2578,-49.2647,
9,Hithadhoo,MV,83.64,overcast clouds,-0.6000,73.0833,
15,Kapaa,US,82.72,light rain,22.0752,-159.3190,
17,Butaritari,KI,83.26,overcast clouds,3.0707,172.7902,
22,Kahului,US,85.96,scattered clouds,20.8947,-156.4700,
24,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,


In [84]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [86]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Galesong,ID,75.22,broken clouds,-5.3166,119.3661,Sapriadi Appy
2,Conakry,GN,84.27,scattered clouds,9.5380,-13.6773,Riviera Royal Hotel
3,Alepe,CI,82.27,broken clouds,5.5004,-3.6631,BENENE 1
5,Gigmoto,PH,85.35,broken clouds,13.7796,124.3900,Cj's Travellers Inn
8,Conceicao Do Araguaia,BR,82.53,clear sky,-8.2578,-49.2647,Tarumã Tropical Hotel
9,Hithadhoo,MV,83.64,overcast clouds,-0.6000,73.0833,Scoop Guest House
15,Kapaa,US,82.72,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
17,Butaritari,KI,83.26,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
22,Kahului,US,85.96,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
24,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [87]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [88]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [89]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))